In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, BatchNormalization, Embedding, SpatialDropout1D

In [2]:
# Load your CSV data
train_file_path = 'RawData_seq/train_seq.csv'
val_file_path = 'RawData_seq/val_seq.csv'

df_train = pd.read_csv(train_file_path)
df_val = pd.read_csv(val_file_path)

In [3]:
# Extract features and labels
X_train = df_train['mfcc'].apply(eval)
y_train = df_train['label']

X_val = df_val['mfcc'].apply(eval)
y_val = df_val['label']

In [4]:
# Encode class labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.fit_transform(y_val)

In [5]:
# Padding sequences to the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_padded = pad_sequences(X_train, dtype='float32', padding='post')
X_train_reshaped = np.expand_dims(X_train_padded, axis=2)

X_val_padded = pad_sequences(X_val, dtype='float32', padding='post')
X_val_reshaped = np.expand_dims(X_val_padded, axis=2)

In [6]:
# ## V1
# model = Sequential()
# model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=(len(X_train_reshaped[0]), 1)))
# model.add(Dropout(0.5))  # Add dropout for regularization

# model.add(Bidirectional(LSTM(128, return_sequences=True)))
# model.add(Dropout(0.5))

# model.add(Bidirectional(LSTM(64, return_sequences=True)))
# model.add(Dropout(0.5))

# model.add(Bidirectional(LSTM(32)))
# model.add(Dropout(0.5))

# model.add(Dense(4, activation='softmax'))

In [7]:
## V2
model = Sequential()

# First Bidirectional LSTM layer with Batch Normalization and Dropout
model.add(Bidirectional(LSTM(512, return_sequences=True), input_shape=(len(X_train_reshaped[0]), 1)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Second Bidirectional LSTM layer with Batch Normalization and Dropout
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Third Bidirectional LSTM layer with Batch Normalization and Dropout
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Fourth Bidirectional LSTM layer with Batch Normalization and Dropout
model.add(Bidirectional(LSTM(64)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(4, activation='softmax'))


In [8]:
# Compile the model
from keras.optimizers import Adam

# Compile the model with sparse_categorical_crossentropy and sparse_categorical_accuracy
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 13, 1024)         2105344   
 l)                                                              
                                                                 
 batch_normalization (BatchN  (None, 13, 1024)         4096      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 13, 1024)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 13, 512)          2623488   
 nal)                                                            
                                                                 
 batch_normalization_1 (Batc  (None, 13, 512)          2048      
 hNormalization)                                        

In [9]:
history = model.fit(X_train_reshaped, y_train_encoded, epochs=15, batch_size=32, validation_data=(X_val_reshaped, y_val_encoded))

Epoch 1/15
497/497 [==============================] - 471s 898ms/step - loss: 1.2382 - sparse_categorical_accuracy: 0.5218 - val_loss: 0.9065 - val_sparse_categorical_accuracy: 0.5824
Epoch 2/15
497/497 [==============================] - 409s 823ms/step - loss: 0.8225 - sparse_categorical_accuracy: 0.6713 - val_loss: 1.0014 - val_sparse_categorical_accuracy: 0.5657
Epoch 3/15
497/497 [==============================] - 363s 731ms/step - loss: 0.7549 - sparse_categorical_accuracy: 0.7019 - val_loss: 0.8785 - val_sparse_categorical_accuracy: 0.6544
Epoch 4/15
497/497 [==============================] - 353s 711ms/step - loss: 0.7305 - sparse_categorical_accuracy: 0.7171 - val_loss: 0.9063 - val_sparse_categorical_accuracy: 0.6227
Epoch 5/15
497/497 [==============================] - 356s 717ms/step - loss: 0.6974 - sparse_categorical_accuracy: 0.7336 - val_loss: 1.1803 - val_sparse_categorical_accuracy: 0.5476
Epoch 6/15
497/497 [==============================] - 382s 769ms/step - loss: 0.

KeyboardInterrupt: 

In [ ]:
# Save model
model.save('Models/slid_rnn_v2_e15.h5')

In [ ]:
test_file_path = 'RawData_seq/test_seq.csv'
df_test = pd.read_csv(test_file_path)

In [ ]:
X_test = df_test['mfcc'].apply(eval)
y_test = df_test['label']

In [ ]:
y_test_encoded = label_encoder.fit_transform(y_test)

X_test_padded = pad_sequences(X_test, dtype='float32', padding='post')
X_test_reshaped = np.expand_dims(X_test_padded, axis=2)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_encoded)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# v1 e15 - 77
# v1 e30 - 79
# v2 e30 - 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

In [ ]:
predictions = model.predict(X_test_reshaped)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = y_test_encoded
conf_matrix = confusion_matrix(true_labels, predicted_labels)

In [ ]:
# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))